# Problem Statement
use machine learning to create a model that predicts which passengers survived the Titanic shipwreck.


In [1]:
# importing libraries
import pandas as pd
import numpy as np

In [2]:
train_data=pd.read_csv("D:\\Kaggle Projects\\titanic\\train.csv")
test_data=pd.read_csv("D:\\Kaggle Projects\\titanic\\test.csv")
train_data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
#Create Deck variable from Cabin at NaNs replace with 'missing'
train_data['Cabin']=train_data['Cabin'].fillna("missing")
Deck={0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'T',9:'missing'}
D1=[]
for i in range(train_data.shape[0]):
    for j in range(len(Deck)):
        if train_data['Cabin'][i].startswith(Deck[j]):
            D1.append(Deck[j])
train_data['Deck']=D1     

In [4]:
#Test data Create Deck variable from Cabin at NaNs replace with 'missing'
test_data['Cabin']=test_data['Cabin'].fillna("missing")
Deck1={0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'T',9:'missing'}
D2=[]
for i in range(test_data.shape[0]):
    for j in range(len(Deck1)):
        if test_data['Cabin'][i].startswith(Deck1[j]):
            D2.append(Deck1[j])
test_data['Deck']=D2 

In [5]:
train_data['Cabin'].value_counts()

missing        687
B96 B98          4
G6               4
C23 C25 C27      4
E101             3
              ... 
D6               1
D46              1
D15              1
C111             1
E77              1
Name: Cabin, Length: 148, dtype: int64

In [6]:
train_data.shape

(891, 13)

Data Dictionary
Variable	Definition	Key
survival	Survival	0 = No, 1 = Yes
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton
Variable Notes
pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

In [7]:
# Fare
train_data['Fare'].mode()

0    8.05
dtype: float64

In [8]:
#Here we will make new feature Title_of_name from Name
f1=train_data['Name'].str.split(" ", n = -1, expand = True)

In [9]:
f1[1].unique()

array(['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Planke,', 'Don.', 'Rev.',
       'Billiard,', 'der', 'Walle,', 'Dr.', 'Pelsmaeker,', 'Mulder,', 'y',
       'Steen,', 'Carlo,', 'Mme.', 'Impe,', 'Ms.', 'Major.', 'Gordon,',
       'Messemaeker,', 'Mlle.', 'Col.', 'Capt.', 'Velde,', 'the',
       'Shawah,', 'Jonkheer.', 'Melkebeke,', 'Cruyssen,'], dtype=object)

In [10]:
#Here convert all in 4 categories Mr , Mrs , Miss , Master
nm_mr=['Mr.','Don.', 'Major.', 'Capt.','Sir.', 'Jonkheer.', 'Rev.', 'Col.']
nm_mrs=['Mrs.','Countess.','Mme.','Lady.','Dona.']
nm_miss=['Miss.','Mlle.', 'Ms.']
title=[]

for i in range(train_data.shape[0]):
    for j in range(len(nm_mr)):
        if nm_mr[j] in train_data['Name'][i]:
            title.append('Mr')
            break
    for j in range(len(nm_mrs)):
        if nm_mrs[j] in train_data['Name'][i]:
            title.append('Mrs')
            break
    for j in range(len(nm_miss)):
        if nm_miss[j] in train_data['Name'][i]:
            title.append('Miss')
            break
    if 'Master.' in train_data['Name'][i]:
        title.append('Master')
    elif 'Dr.' in train_data['Name'][i]:
        if train_data['Sex'][i]=='Male':
            title.append('Mr')
        else:
            title.append('Mrs')
#creating new feature Title_of_name in dataset
train_data['Title_of_name']=title
train_data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,Title_of_name
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,missing,S,missing,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,missing,S,missing,Miss


In [11]:
"""
#Used this code to fnd some titles which are not founded in names
index=[]
title_list=['Mrs.', 'Mr.', 'Master.', 'Miss.', 'Major.', 'Rev.',
                    'Dr.', 'Ms.', 'Mlle.','Col.', 'Capt.', 'Mme.', 'Countess.',
                    'Don.', 'Jonkheer.']
for i in range(train_data.shape[0]):
    flag=True
    for j in range(len(title_list)):
        if title_list[j] in train_data['Name'][i]:
            flag=False
            break
    if flag==True:
        index.append(i)
for i in index:
    print(train_data['Name'][i])         
"""         

"\n#Used this code to fnd some titles which are not founded in names\nindex=[]\ntitle_list=['Mrs.', 'Mr.', 'Master.', 'Miss.', 'Major.', 'Rev.',\n                    'Dr.', 'Ms.', 'Mlle.','Col.', 'Capt.', 'Mme.', 'Countess.',\n                    'Don.', 'Jonkheer.']\nfor i in range(train_data.shape[0]):\n    flag=True\n    for j in range(len(title_list)):\n        if title_list[j] in train_data['Name'][i]:\n            flag=False\n            break\n    if flag==True:\n        index.append(i)\nfor i in index:\n    print(train_data['Name'][i])         \n"

In [12]:
train_data['SibSp'].value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [13]:
train_data.dtypes

PassengerId        int64
Survived           int64
Pclass             int64
Name              object
Sex               object
Age              float64
SibSp              int64
Parch              int64
Ticket            object
Fare             float64
Cabin             object
Embarked          object
Deck              object
Title_of_name     object
dtype: object

In [14]:
train_data['Parch'].value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

In [15]:
train_data.shape

(891, 14)

In [16]:
#Here 0 means Not survived and 1 means survived
train_data['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [17]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [18]:
#null values checking and removing
train_data.isnull().sum()

PassengerId        0
Survived           0
Pclass             0
Name               0
Sex                0
Age              177
SibSp              0
Parch              0
Ticket             0
Fare               0
Cabin              0
Embarked           2
Deck               0
Title_of_name      0
dtype: int64

In [19]:
#We will remove the Cabin column and we will impute Age and Embarked column
train_data.drop(columns=['Cabin','Name','Ticket'],axis=1,inplace=True)
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked', 'Deck', 'Title_of_name'],
      dtype='object')

In [20]:
#Now fill Embarked
train_data['Embarked']=train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])

In [21]:
print(train_data['Embarked'].unique())
print(train_data['Sex'].unique())
print(train_data['Deck'].unique())

['S' 'C' 'Q']
['male' 'female']
['missing' 'C' 'E' 'G' 'D' 'A' 'B' 'F' 'T']


In [22]:
#First we need to convert categorical data into numeric then we will apply the Imputer on age
train_data.loc[train_data['Sex']=='male','Sex']=0
train_data.loc[train_data['Sex']=='female','Sex']=1

train_data.loc[train_data['Embarked']=='S','Embarked']=0
train_data.loc[train_data['Embarked']=='C','Embarked']=1
train_data.loc[train_data['Embarked']=='Q','Embarked']=2

train_data.loc[train_data['Title_of_name']=='Mr','Title_of_name']=0
train_data.loc[train_data['Title_of_name']=='Mrs','Title_of_name']=1
train_data.loc[train_data['Title_of_name']=='Miss','Title_of_name']=2
train_data.loc[train_data['Title_of_name']=='Master','Title_of_name']=3

train_data.loc[train_data['Deck']=='missing','Deck']=0
train_data.loc[train_data['Deck']=='A','Deck']=1
train_data.loc[train_data['Deck']=='B','Deck']=2
train_data.loc[train_data['Deck']=='C','Deck']=3
train_data.loc[train_data['Deck']=='D','Deck']=4
train_data.loc[train_data['Deck']=='E','Deck']=5
train_data.loc[train_data['Deck']=='F','Deck']=6
train_data.loc[train_data['Deck']=='G','Deck']=7
train_data.loc[train_data['Deck']=='T','Deck']=8


In [23]:
train_data.head(2)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Title_of_name
0,1,0,3,0,22.0,1,0,7.2500,0,0,0
1,2,1,1,1,38.0,1,0,71.2833,1,3,1


In [24]:
#Handling missing values

In [25]:
#Do this step after converting data into numeric format
from sklearn.impute import KNNImputer
impute_knn=KNNImputer(n_neighbors=3)
new_d=impute_knn.fit_transform(train_data)
train1=pd.DataFrame(new_d,columns=['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare','Deck','Embarked','Title_of_name'])

In [26]:
train1.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Deck,Embarked,Title_of_name
0,1.0,0.0,3.0,0.0,22.0,1.0,0.0,7.2500,0.0,0.0,0.0
1,2.0,1.0,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,3.0,1.0
2,3.0,1.0,3.0,1.0,26.0,0.0,0.0,7.9250,0.0,0.0,2.0


In [27]:
train1.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Deck             0
Embarked         0
Title_of_name    0
dtype: int64

# We can create another feature from Age
# Baby 0-2   Children 2-16  young Adult 16-40  middle age adult= 40-60  Old >60

In [28]:

age1=[]
for i in range(train1.shape[0]):
    if train1['Age'][i] < 2.0:
        age1.append('baby')
    elif train1['Age'][i] >= 2.0 and train1['Age'][i]< 16.0:
        age1.append('children')
    elif train1['Age'][i] >= 16.0 and train1['Age'][i]< 40.0:
        age1.append('young adult')
    elif train1['Age'][i] >= 40.0 and train1['Age'][i]< 60.0:
        age1.append('middle age')
    elif train1['Age'][i] >= 60.0:
        age1.append('old')
train1['Age_type']=age1

In [29]:
train1.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Deck,Embarked,Title_of_name,Age_type
0,1.0,0.0,3.0,0.0,22.0,1.0,0.0,7.2500,0.0,0.0,0.0,young adult
1,2.0,1.0,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,3.0,1.0,young adult
2,3.0,1.0,3.0,1.0,26.0,0.0,0.0,7.9250,0.0,0.0,2.0,young adult


# TicketClass variable from Fare

In [30]:
#here 1 for first class and 3 for 3rd class
Tk1=[]
for i in range(train1.shape[0]):
    if train1['Fare'][i] <=40.0:
        Tk1.append(1)
    elif train1['Fare'][i] >40.0 and train1['Fare'][i]<=80.0:
        Tk1.append(2)
    else:
        Tk1.append(3)
train1['Fare_class']=Tk1
train1.head(2)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Deck,Embarked,Title_of_name,Age_type,Fare_class
0,1.0,0.0,3.0,0.0,22.0,1.0,0.0,7.2500,0.0,0.0,0.0,young adult,1
1,2.0,1.0,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,3.0,1.0,young adult,2


In [31]:
train1.loc[train1['Age_type']=='baby','Age_type']=1
train1.loc[train1['Age_type']=='children','Age_type']=2
train1.loc[train1['Age_type']=='young adult','Age_type']=3
train1.loc[train1['Age_type']=='middle age','Age_type']=4
train1.loc[train1['Age_type']=='old','Age_type']=5
train1['Age_type']=train1['Age_type'].astype('int64')
train1.head(2)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Deck,Embarked,Title_of_name,Age_type,Fare_class
0,1.0,0.0,3.0,0.0,22.0,1.0,0.0,7.2500,0.0,0.0,0.0,3,1
1,2.0,1.0,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,3.0,1.0,3,2


In [32]:
train1.corr()['Survived']

PassengerId     -0.005007
Survived         1.000000
Pclass          -0.338481
Sex              0.543351
Age             -0.064920
SibSp           -0.035322
Parch            0.081629
Fare             0.257307
Deck             0.106811
Embarked         0.287944
Title_of_name    0.473751
Age_type        -0.096660
Fare_class       0.281688
Name: Survived, dtype: float64

In [33]:
# feature selection

In [34]:
X=train1.drop(['PassengerId','Survived'],axis=1)
y=train1['Survived']

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=50)
print(X_train.shape)
print(y_train.shape)

(623, 11)
(623,)


In [37]:
#Here for removing outliers we use RobustScaler
from sklearn.preprocessing import RobustScaler

In [38]:
transformer_robust = RobustScaler()

In [39]:
robust_x_train = transformer_robust.fit_transform(X_train)

In [40]:
robust_x_test=transformer_robust.transform(X_test)

In [41]:
import xgboost as xgb

In [42]:
xg_model1=xgb.XGBClassifier(colsample_bytree = 0.35, learning_rate = 0.05,
                max_depth =8, n_estimators =100,n_jobs=5)

In [43]:
xg_model1.fit(robust_x_train,y_train)
pred1=xg_model1.predict(robust_x_test)

D:\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:07:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [44]:
from sklearn.metrics import accuracy_score

In [45]:
accuracy_score(pred1,y_test)

0.832089552238806

In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
LR=LogisticRegression(C=5,random_state=30)

In [48]:
LR.fit(robust_x_train,y_train)
pred2=LR.predict(robust_x_test)

In [49]:
accuracy_score(pred2,y_test)

0.8134328358208955

In [50]:
from sklearn.neighbors import KNeighborsClassifier

In [51]:
knn=KNeighborsClassifier(n_neighbors=9)

In [52]:
knn.fit(robust_x_train,y_train)
pred3=knn.predict(robust_x_test)

In [53]:
accuracy_score(pred3,y_test)

0.835820895522388

In [54]:
from sklearn.ensemble import RandomForestClassifier

In [55]:
rf=RandomForestClassifier(n_estimators=150,max_depth=6,min_samples_split=4,random_state=80)
rf.fit(robust_x_train,y_train)
pred4=rf.predict(robust_x_test)

In [56]:
accuracy_score(pred4,y_test)

0.835820895522388

In [57]:
# Preprocess the test data

In [58]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,missing,Q,missing
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,missing,S,missing
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,missing,Q,missing
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,missing,S,missing
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,missing,S,missing


In [59]:
test_data.shape

(418, 12)

In [60]:
#Here we will make new feature Title_of_name from Name
f2=test_data['Name'].str.split(" ", n = -1, expand = True)
f2[1].unique()

array(['Mr.', 'Mrs.', 'Miss.', 'Carlo,', 'Khalil,', 'Master.', 'y', 'Ms.',
       'Palmquist,', 'Col.', 'Planke,', 'Rev.', 'Billiard,',
       'Messemaeker,', 'Dr.', 'Brito,'], dtype=object)

In [61]:
title1=[]

for i in range(test_data.shape[0]):
    for j in range(len(nm_mr)):
        if nm_mr[j] in test_data['Name'][i]:
            title1.append('Mr')
            break
    for j in range(len(nm_mrs)):
        if nm_mrs[j] in test_data['Name'][i]:
            title1.append('Mrs')
            break
    for j in range(len(nm_miss)):
        if nm_miss[j] in test_data['Name'][i]:
            title1.append('Miss')
            break
    if 'Master.' in test_data['Name'][i]:
        title1.append('Master')
    elif 'Dr.' in test_data['Name'][i]:
        if test_data['Sex'][i]=='Male':
            title1.append('Mr')
        else:
            title1.append('Mrs')
#creating new feature Title_of_name in dataset
test_data['Title_of_name']=title1
test_data.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,Title_of_name
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,missing,Q,missing,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,missing,S,missing,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,missing,Q,missing,Mr


In [62]:
test_data.dtypes

PassengerId        int64
Pclass             int64
Name              object
Sex               object
Age              float64
SibSp              int64
Parch              int64
Ticket            object
Fare             float64
Cabin             object
Embarked          object
Deck              object
Title_of_name     object
dtype: object

In [63]:
test_data.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age              86
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin             0
Embarked          0
Deck              0
Title_of_name     0
dtype: int64

In [64]:
#removing unnecessary columns
test_data.drop(columns=['Cabin','Name','Ticket'],axis=1,inplace=True)
test_data.columns

Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'Deck', 'Title_of_name'],
      dtype='object')

In [65]:
#converting categorical data into numerical

In [66]:
print(test_data['Embarked'].unique())
print(test_data['Sex'].unique())
print(test_data['Title_of_name'].unique())

['Q' 'S' 'C']
['male' 'female']
['Mr' 'Mrs' 'Miss' 'Master']


In [67]:
#First we need to convert categorical data into numeric then we will apply the Imputer on age
test_data.loc[test_data['Sex']=='male','Sex']=0
test_data.loc[test_data['Sex']=='female','Sex']=1

test_data.loc[test_data['Embarked']=='S','Embarked']=0
test_data.loc[test_data['Embarked']=='C','Embarked']=1
test_data.loc[test_data['Embarked']=='Q','Embarked']=2

test_data.loc[test_data['Title_of_name']=='Mr','Title_of_name']=0
test_data.loc[test_data['Title_of_name']=='Mrs','Title_of_name']=1
test_data.loc[test_data['Title_of_name']=='Miss','Title_of_name']=2
test_data.loc[test_data['Title_of_name']=='Master','Title_of_name']=3


test_data.loc[test_data['Deck']=='missing','Deck']=0
test_data.loc[test_data['Deck']=='A','Deck']=1
test_data.loc[test_data['Deck']=='B','Deck']=2
test_data.loc[test_data['Deck']=='C','Deck']=3
test_data.loc[test_data['Deck']=='D','Deck']=4
test_data.loc[test_data['Deck']=='E','Deck']=5
test_data.loc[test_data['Deck']=='F','Deck']=6
test_data.loc[test_data['Deck']=='G','Deck']=7
test_data.loc[test_data['Deck']=='T','Deck']=8


In [68]:
test_data.head(2)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Title_of_name
0,892,3,0,34.5,0,0,7.8292,2,0,0
1,893,3,1,47.0,1,0,7.0000,0,0,1


In [69]:
#Null fill null values with KNNImputer
from sklearn.impute import KNNImputer
impute_knn1=KNNImputer(n_neighbors=3)
new_d1=impute_knn1.fit_transform(test_data)
test1=pd.DataFrame(new_d1,columns=['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare','Deck','Embarked','Title_of_name'])

In [70]:
age2=[]
for i in range(test1.shape[0]):
    if test1['Age'][i] < 2.0:
        age2.append('baby')
    elif test1['Age'][i] >= 2.0 and test1['Age'][i]< 16.0:
        age2.append('children')
    elif test1['Age'][i] >= 16.0 and test1['Age'][i]< 40.0:
        age2.append('young adult')
    elif test1['Age'][i] >= 40.0 and test1['Age'][i]< 60.0:
        age2.append('middle age')
    elif test1['Age'][i] >= 60.0:
        age2.append('old')
test1['Age_type']=age2

In [71]:
#here 1 for first class and 3 for 3rd class
Tk2=[]
for i in range(test1.shape[0]):
    if test1['Fare'][i] <=40.0:
        Tk2.append(1)
    elif test1['Fare'][i] >40.0 and test1['Fare'][i]<=80.0:
        Tk2.append(2)
    else:
        Tk2.append(3)
test1['Fare_class']=Tk2
test1.head(2)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Deck,Embarked,Title_of_name,Age_type,Fare_class
0,892.0,3.0,0.0,34.5,0.0,0.0,7.8292,2.0,0.0,0.0,young adult,1
1,893.0,3.0,1.0,47.0,1.0,0.0,7.0000,0.0,0.0,1.0,middle age,1


In [72]:
test1.loc[test1['Age_type']=='baby','Age_type']=1
test1.loc[test1['Age_type']=='children','Age_type']=2
test1.loc[test1['Age_type']=='young adult','Age_type']=3
test1.loc[test1['Age_type']=='middle age','Age_type']=4
test1.loc[test1['Age_type']=='old','Age_type']=5
test1['Age_type']=test1['Age_type'].astype('int64')
test1.head(2)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Deck,Embarked,Title_of_name,Age_type,Fare_class
0,892.0,3.0,0.0,34.5,0.0,0.0,7.8292,2.0,0.0,0.0,3,1
1,893.0,3.0,1.0,47.0,1.0,0.0,7.0000,0.0,0.0,1.0,4,1


In [73]:
test1.isnull().sum()

PassengerId      0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Deck             0
Embarked         0
Title_of_name    0
Age_type         0
Fare_class       0
dtype: int64

In [74]:
test1.drop(['PassengerId'],axis=1,inplace=True)

In [75]:
#transform data using robust scaling 
test1_robust=transformer_robust.transform(test1)

In [76]:
#final_xgb=xg_model1.predict(test1_robust)
#final_xgb=final_xgb.astype('int64')

# Creating Submission File

In [77]:
"""
# create sample submission file and submit
predict1=pd.DataFrame(final_xgb)
#sub_df1=pd.read_csv('SAMPLE.csv')
dataset1=pd.concat([test_data['PassengerId'],predict1],axis=1)
dataset1.columns=['PassengerId','Survived']
dataset1.to_csv('xgb1_submission.csv',index=False)
"""

"\n# create sample submission file and submit\npredict1=pd.DataFrame(final_xgb)\n#sub_df1=pd.read_csv('SAMPLE.csv')\ndataset1=pd.concat([test_data['PassengerId'],predict1],axis=1)\ndataset1.columns=['PassengerId','Survived']\ndataset1.to_csv('xgb1_submission.csv',index=False)\n"

In [78]:
final_rf=rf.predict(test1_robust)
final_rf=final_rf.astype('int64')

In [79]:
# create sample submission file and submit
predict2=pd.DataFrame(final_rf)
dataset2=pd.concat([test_data['PassengerId'],predict2],axis=1)
dataset2.columns=['PassengerId','Survived']
dataset2.to_csv('rf1_morefeaures_submission.csv',index=False)


In [80]:
#here I used voting classifier it will take voting of multiple models 
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

In [81]:
vc=VotingClassifier([('m1',rf),('m2',LR),('m3',knn)],voting='soft')
cross_val_score(vc,robust_x_train,y_train).mean()

0.8186322580645161

In [82]:
vc.fit(robust_x_train,y_train)
vc_pred=vc.predict(X_test)

In [83]:
accuracy_score(vc_pred,y_test)

0.7089552238805971

In [84]:
#final_LR=LR.predict(test1_robust)
#final_LR=final_LR.astype('int64')

In [85]:
# create sample submission file and submit
#predict2=pd.DataFrame(final_LR)
#dataset2=pd.concat([test_data['PassengerId'],predict2],axis=1)
#dataset2.columns=['PassengerId','Survived']
#dataset2.to_csv('LR_submission.csv',index=False)

In [86]:
#final_vc=vc.predict(test1_robust)
#final_vc=final_vc.astype('int64')

In [87]:
#create sample submission file and submit
#predict2=pd.DataFrame(final_vc)
#dataset2=pd.concat([test_data['PassengerId'],predict2],axis=1)
#dataset2.columns=['PassengerId','Survived']
#dataset2.to_csv('VC_submission.csv',index=False)

In [88]:
# 